In [2]:
import pandas as pd
import numpy as np
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC 
from selenium.webdriver.common.action_chains import ActionChains
from io import StringIO
import re
from scipy.stats import zscore
from scipy.stats import ttest_ind
from multiprocessing import Pool

In [3]:
driver   = webdriver.Firefox(executable_path = r'C:\Users\AbdallahAmr\Documents\ProgrammingFiles\geckodriver.exe')
driver.maximize_window()

driver.get("http://asd.princeton.edu/") 



WebDriverException: Message: 'geckodriver.exe' executable needs to be in PATH. 


In [283]:
f = open("demo_fnu_copy.txt", "a")
for i in range(1110):
    string = ''
    wait   = WebDriverWait(driver,10)
    table  = wait.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, '#genomePredTable')))
    element = wait.until(EC.visibility_of_all_elements_located((By.CSS_SELECTOR, "#genomePredTable_paginate > ul:nth-child(1) > li:nth-child(8) > a:nth-child(1)")))
    element = element.pop()
    driver.execute_script("arguments[0].scrollIntoView();", element)
    driver.execute_script("arguments[0].click();", element)

    
#     actions = ActionChains(driver)
#     actions.move_to_element(element)
#     actions.perform()
#     actions.click(element)
#     actions.perform()
#     wait.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, '#genomePredTable')))
    #print(table.pop().text)
    for tab in table:
        #koko = '21 AFF2 1 0.25 0.814 0.001 0.0113 0 0 0 0 0 0'
        lines = tab.text.splitlines()
        for i in lines[1:]:
            R = re.search(r'[\d]+ ',i).group(0)
            N = re.search(r' [\w\d]+ ',i).group(0)
            #print(i)
            prob = re.search(r' 0.[9876543][1234567890 ][\d ][ ]*',i).group(0)
            one_line = R+' '+N+' '+prob
            f.write(one_line+'\n')
#             string = string + '\n'+one_line
        
    #list1.append(string)
    

In [2]:
ASD_ = pd.read_csv('demo11.txt',names = ['rank','gene','prob'],delimiter='   ',index_col='rank')

D:\ProgramFiles\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  """Entry point for launching an IPython kernel.


In [3]:
data_df = pd.read_csv('pre_CDA.txt',delimiter='\t')

In [4]:
col_list = []
for k in range(69):
    if k == 0:
        col_list.append('0')
        continue
    col_list.append('0.%d'%k)

In [5]:
data_df = data_df.drop(axis=1,columns=col_list)

In [22]:
first_300 = list(ASD_.gene[0:300])
last_300 = list(ASD_.gene[21884-351:])

In [23]:
df_list = []
list_of_shared_genes1 = data_df.loc[data_df['IDENTIFIER'].isin(first_300)]
list_of_shared_genes2 = data_df.loc[data_df['IDENTIFIER'].isin(last_300)]

In [24]:
list_of_shared_genes = pd.concat([list_of_shared_genes1,list_of_shared_genes2])

In [25]:
new_list_shared = list_of_shared_genes.set_index(list_of_shared_genes.IDENTIFIER)

In [26]:
very_new_list_shared = new_list_shared.drop(['IDENTIFIER'],axis=1)

In [27]:
normalized_genes = very_new_list_shared

In [30]:
labels = ['mean_A','mean_B','std_A','std_B','rng_A','rng_B','ttest','PCC','prob_A','prob_B']
list_of_ser = []
for i in range(len(normalized_genes)):
    c = i+1
    print(i)
    for c in range(len(normalized_genes)):

        list_of_vals = []
        list_A = list(normalized_genes.iloc[i])
        list_B = list(normalized_genes.iloc[c])

        mean_A = np.mean(list_A)
        mean_B = np.mean(list_B)
        std_A  = np.std(list_A)
        std_B  = np.std(list_B)
        PCC    = np.corrcoef(list_A,list_B)
        prob_A = ASD_.loc[ASD_.gene==normalized_genes.iloc[i].name,'prob'].iloc[0]
        prob_B = ASD_.loc[ASD_.gene==normalized_genes.iloc[c].name,'prob'].iloc[0]
        rng_A  = np.amax(list_A) - np.amin(list_A)
        rng_B  = np.amax(list_B) - np.amin(list_B)
        ttest  = ttest_ind(list_A,list_B) 
        
#         pool = Pool()      
#         result1 = pool.apply_async(np.mean, [list_A])
#         result2 = pool.apply_async(np.mean, [list_B])
#         result3 = pool.apply_async(np.std, [list_A])
#         result4 = pool.apply_async(np.std, [list_B])
#         result5  = pool.apply_async(np.amax, [list_A])
#         result5_ = pool.apply_async(np.amin, [list_A])     
#         result6  = pool.apply_async(np.amax, [list_B])
#         result6_ = pool.apply_async(np.amin, [list_B])
#         result7 = pool.apply_async(ttest_ind, [list_A,list_B])
#         result8 = pool.apply_async(np.corrcoef, [list_A,list_B])
        
         
       
        
        
#         mean_A = result1.get(timeout=10)
#         mean_B = result2.get(timeout=10)
#         std_A  = result3.get(timeout=10)
#         std_B  = result4.get(timeout=10)
#         rng_A  = result5.get(timeout=10)-result5_.get(timeout=10)
#         rng_B  = result6.get(timeout=10)-result6_.get(timeout=10)
#         ttest  = result7.get(timeout=10)
#         PCC    = result8.get(timeout=10)
        
        list_of_vals = [mean_A,mean_B,std_A,std_B,rng_A,rng_B,ttest,PCC,prob_A,prob_B]
        DD = pd.Series(list_of_vals,index=labels,name=normalized_genes.iloc[i].name+'_'+normalized_genes.iloc[c].name)
        list_of_ser.append(DD)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [31]:
import pickle


In [32]:
pickle.dump( list_of_ser, open( "save.p", "wb" ) )

In [33]:
len(list_of_ser)

1425636

In [36]:
df_pre_model = pd.concat(list_of_ser[0:1000],axis=1)

In [39]:
list_of_ser[0:10]

[mean_A                   -0.320707
 mean_B                   -0.320707
 std_A                    0.0250922
 std_B                    0.0250922
 rng_A                    0.0984391
 rng_B                    0.0984391
 ttest                   (0.0, 1.0)
 PCC       [[1.0, 1.0], [1.0, 1.0]]
 prob_A                       0.781
 prob_B                       0.781
 Name: NAV3_NAV3, dtype: object,
 mean_A                                            -0.320707
 mean_B                                            -0.310757
 std_A                                             0.0250922
 std_B                                             0.0251377
 rng_A                                             0.0984391
 rng_B                                              0.110183
 ttest            (-2.4422007002077435, 0.01574483377355135)
 PCC       [[1.0, 0.9870590411114564], [0.987059041111456...
 prob_A                                                0.781
 prob_B                                                0.7